In [17]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=5000)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('GME', 
                      start=start_date, 
                      end=end_date, 
                      progress=False)
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close", 
             "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)
data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
3441,2024-05-13,26.340000,38.200001,24.770000,30.450001,30.450001,187241700
3442,2024-05-14,64.830002,64.830002,36.000000,48.750000,48.750000,206979100
3443,2024-05-15,40.310001,42.349998,31.000000,39.549999,39.549999,131790100
3444,2024-05-16,33.980000,35.240002,27.590000,27.670000,27.670000,76177600
3445,2024-05-17,21.860001,22.410000,19.700001,22.209999,22.209999,94564600


In [18]:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"], 
                                        high=data["High"],
                                        low=data["Low"], 
                                        close=data["Close"])])
figure.update_layout(title = "GameStop Stock Price Analysis", 
                     xaxis_rangeslider_visible=False)
figure.show()

In [19]:
correlation = data.corr()
print(correlation["Close"].sort_values(ascending=False))

Close        1.000000
Adj Close    0.996361
Low          0.996195
Open         0.994670
High         0.992248
Volume       0.108481
Name: Close, dtype: float64


In [20]:
x = data[["Open", "High", "Low", "Volume"]]
y = data["Close"]
x = x.to_numpy()
y = y.to_numpy()
y = y.reshape(-1, 1)

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

In [21]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (xtrain.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 4, 128)            66560     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_4 (Dense)             (None, 25)                1625      
                                                                 
 dense_5 (Dense)             (None, 1)                 26        
                                                                 
Total params: 117,619
Trainable params: 117,619
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(xtrain, ytrain, batch_size=1, epochs=50)

Epoch 1/50
2756/2756 [==============================] - 20s 6ms/step - loss: 0.8289
Epoch 2/50
2756/2756 [==============================] - 16s 6ms/step - loss: 0.7841
Epoch 3/50
2756/2756 [==============================] - 16s 6ms/step - loss: 0.5772
Epoch 4/50
2756/2756 [==============================] - 16s 6ms/step - loss: 0.8795
Epoch 5/50
2756/2756 [==============================] - 18s 6ms/step - loss: 0.8093
Epoch 6/50
2756/2756 [==============================] - 16s 6ms/step - loss: 0.6847
Epoch 7/50
2756/2756 [==============================] - 17s 6ms/step - loss: 0.7474
Epoch 8/50
2756/2756 [==============================] - 16s 6ms/step - loss: 0.9263
Epoch 9/50
2756/2756 [==============================] - 17s 6ms/step - loss: 0.7755
Epoch 10/50
2756/2756 [==============================] - 18s 6ms/step - loss: 0.7863
Epoch 11/50
2756/2756 [==============================] - 16s 6ms/step - loss: 0.7178
Epoch 12/50
2756/2756 [==============================] - 17s 6ms/step - lo

In [25]:
import numpy as np
#features = [Open, High, Low, Adj Close, Volume]
features = np.array([[26.340000, 38.200001, 24.770000, 187241700]])
model.predict(features)

array([[28.566284]], dtype=float32)